In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
from google.colab import files
import gspread
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe
import numpy as np
import pandas as pd
from dateutil.parser import parse

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
uploaded = files.upload()

In [ ]:
input_first_date = '2025-06-16'
input_last_date = '2025-06-21'

In [ ]:
filename = next(iter(uploaded))
CT_dashboard_df = pd.read_excel(filename)
CT_dashboard_df = CT_dashboard_df[CT_dashboard_df.iloc[:, 1].notna()]
CT_dashboard_df["Check 1"] = CT_dashboard_df["Total Waktu Di Jalan > 240 Mins"].apply(lambda x: True if x == "✅" else False)
CT_dashboard_df["Check 2"] = CT_dashboard_df["Mulai > 08:45"].apply(lambda x: True if x == "✅" else False)
CT_dashboard_df["Check 3"] = CT_dashboard_df["Selesai < 15:00 (untuk Sabtu, <13:30)"].apply(lambda x: True if x == "✅" else False)
CT_dashboard_df["Check 4"] = CT_dashboard_df["Durasi Kerja < 360 Mins (untuk Sabtu, <300 Min)"].apply(lambda x: True if x == "✅" else False)
CT_dashboard_df["Check 5"] = CT_dashboard_df["Titik Toko > 5 Di Satu Tempat"].apply(lambda x: True if x == "✅" else False)
CT_dashboard_df["Check 6"] = CT_dashboard_df["Foto Selfie Buram / Blank"].apply(lambda x:True if x == "✅" else False )
CT_dashboard_df["Salesman Id"] = CT_dashboard_df["Salesman Id"].astype(int)

In [ ]:
spreadsheet_name = ["R01_SUMUT_CT_2.0", "R02_SUMBAR_CT_2.0", "R03_SUMTENG_CT_2.0", "R04_SUMSEL_CT_2.0", "R05_BANTEN_CT_2.0", "R06_JABODEBEK_CT_2.0", "R07_BANDUNGSUCI_CT_2.0", "R08_PANTURA_CT_2.0", "R09_PRIANGAN TIMUR_CT_2.0"
, "R10_JATENG UTARA_CT_2.0", "R11_JATENG SELATAN_CT_2.0", "R12_JATIM UTARA_CT_2.0", "R13_JATIM TENGAH_CT_2.0", "R14_JATIM SELATAN_CT_2.0", "R15_BALI NUSRA_CT_2.0", "R16_KALBARTIM_CT_2.0", "R17_KALTENGSEL_CT_2.0", "R18_SULAWESI 1_CT_2.0", "R19_SULAWESI 2_CT_2.0", "R20_SULAWESI 3_CT_2.0"
]

# Mapping Bahasa month names to English
month_map = {
  'Jan': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Apr', 'Mei': 'May',
  'Jun': 'Jun', 'Jul': 'Jul', 'Agu': 'Aug', 'Sep': 'Sep', 'Okt': 'Oct',
  'Nov': 'Nov', 'Des': 'Dec'
}

def translate_month(date_str):
    for indo, eng in month_map.items():
        if indo in date_str:
            return date_str.replace(indo, eng)
    return date_str

def safe_parse(val):
    try:
        translated = translate_month(val)
        return parse(translated, dayfirst=True)
    except (ValueError, TypeError):
        return val

for name in spreadsheet_name:
    spreadsheet = gc.open(name)
    sheet = spreadsheet.worksheet("Pengerjaan Harian - ASS")
    df = get_as_dataframe(sheet, evaluate_formulas=True)
    df.loc[:, 'Unnamed: 2'] = df["Unnamed: 2"].apply(safe_parse)
    merged = pd.merge(
      df,
      CT_dashboard_df[['Date', 'Salesman Id', 'Check 1', 'Check 2', 'Check 3', 'Check 4', 'Check 5', 'Check 6']],
      left_on=['Unnamed: 2', 'Unnamed: 7'],
      right_on=['Date', 'Salesman Id'],
      how='left'
    )

    df.loc[:, 'Unnamed: 2'] = pd.to_datetime(df['Unnamed: 2'], errors='coerce')

    merged['Unnamed: 2'] = pd.to_datetime(merged['Unnamed: 2'], errors='coerce')
    df2 = df[
      (df['Unnamed: 2'] >= pd.to_datetime(input_first_date)) &
      (df['Unnamed: 2'] <= pd.to_datetime(input_last_date))
    ]

    merged2 = merged[
      (merged['Unnamed: 2'] >= pd.to_datetime(input_first_date)) &
      (merged['Unnamed: 2'] <= pd.to_datetime(input_last_date))
    ]
    df2.reset_index(drop=True, inplace=True)
    merged2.reset_index(drop=True, inplace=True)

    df2.loc[:, 'Unnamed: 9'] = merged2["Check 1"]
    df2.loc[:, 'Unnamed: 10'] = merged2["Check 2"]
    df2.loc[:, 'Unnamed: 11'] = merged2["Check 3"]
    df2.loc[:, 'Unnamed: 12'] = merged2["Check 4"]
    df2.loc[:, 'Unnamed: 13'] = merged2["Check 5"]
    df2.loc[:, 'Unnamed: 14'] = merged2["Check 6"]

    row_index = df[df['Unnamed: 2'] == pd.to_datetime(input_first_date)].index.min()
    first_row = row_index+2
    last_row = row_index+2-1+len(df2)
    data_to_update = df2.loc[:, ['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14']]
    data_to_update = data_to_update.replace([np.nan, np.inf, -np.inf], False)
    values = data_to_update.values.tolist()
    sheet.update(f'J{first_row}:O{last_row}', values)
    emailAddress = "digitalizationpma@gmail.com"
    sheetId = sheet.id
    body = {
      "requests": [
        {
            "addProtectedRange": {
                "protectedRange": {
                    "range": {
                        "sheetId": sheetId,
                        "startColumnIndex": 9,
                        "endColumnIndex": 15
                    },
                    "editors": {
                        "domainUsersCanEdit": False,
                        "users": [emailAddress]
                    },
                    "warningOnly": False
                }
            }
        }
      ]
    }
    spreadsheet.batch_update(body)
    print(name)
